# Práctica 2
## Parte 2. Recuperación de Información
<br>

__Alumnos:__
* __Frederick Ernesto Borges Noronha__
* __Victor Manuel Cavero Gracia__

Aquí vamos a utilizar el conjunto de datos de 20 Newsgroups que se encuentra disponible en Scikit-learn y que hemos usado en el notebook de ejemplo.

El conjunto consiste en textos de un foro sobre diferentes temas, desde hardware hasta religión. Algunos temas están muy relacionados, p.ej. "IBM PC hardware" y "Mac hardware", mientras que otros son más diversos, p.ej. "religion" o "hockey").

El objetivo de esta parte es poner en poner en práctica los conceptos de recuperación de información para realizar un buscador de mensajes en un foro.

### Apartado A

El conjunto está dividido de forma predeterminada en entrenamiento y prueba en porcentajes de 60 y 40, respectivamente (como se puede ver en el notebook de prueba). Usaremos únicamente la parte de entrenamiento como los mensajes a recuperar por nuestro buscador.

Vamos a utilizar una representación de la bolsa de palabras de **countVectorizer** con las siguientes opciones:
- La bolsa de palabras tendrá en cuenta la frecuencia de las palabras en cada mensaje (**binary=False**)
- Usa el diccionario que se encuentra en la siguiente URL y que ya usamos en el notebook de prueba. https://github.com/dwyl/english-words/blob/master/words.txt
- Usa la lista de palabras vacías (parámetro **stop_words**) que proporciona sklearn para el inglés
- Usando un rango de n-gramas de (1,1) (parámetro **ngram_range**).

Para calcular la similitud entre dos mensajes usaremos la similitud del coseno (**sklearn.metrics.pairwise.cosine_similarity**) que es capaz de medir la similitud entre los elementos (es decir, entre las filas) de dos matrices de vectores de términos pudiendo ser estas matrices densas o dispersas.

Toma 3 mensaje del conjunto de prueba para cada clase (es decir, para cada tema). Vas a usar cada uno de dichos mensajes como consulta para recuperar los mensajes del conjunto de entrenamiento que más se parezcan a la consulta. Para ello sigue los siguientes pasos:

1. Usa la distancia del coseno entre el mensaje de consulta y los mensajes de entrenamiento.

2. Ordena los resultados de mayor a menor relevancia con la consulta.

3. Calcula la precisión de la lista de resultados con nivel de exhaustividad 3 y 10. 

    - La precisión a un nivel de exhaustividad X es el número de resultados que son relevantes (es decir, de la clase buscada) de entre los X primeros recuperados.

4. Calcula los valores de precisión media (para cada nivel de exhaustividad) para cada clase del conjunto de datos.

Se valorará el uso de funciones y la claridad del código, así como sus comentarios. Contesta a lo siguiente:

- ¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?

- Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.

    - ¿Con qué clases se ha confundido más dicha consulta? 

    - ¿A qué crees que se deben los malos resultados?

Debes usar la parte de entrenamiento para construir la bolsa de palabras con frecuencia y bolsa de palabras con TF/IDF.

### Apartado B

Repite la secuencia de pasos descritos en el apartado a) pero ahora usa TF-IDF para ponderar el peso de los términos de la bolsa de palabras.  Para usar TF-IDF primero debes transformar los textos  usando  **countVectorizer** con **binary=False**  para  obtener  la  frecuencia  de  palabras  (exactamente igual que en el apartado anterior), y a continuación usar **TfidfTransformer** para modular dicha frecuencia según lo popular que sea cada término en el conjunto de mensajes de entrenamiento.

A continuación contesta a lo siguiente.

- ¿Han cambiado los valores de precisión media para las clases del conjunto de datos?¿Qué clases han mejorado? ¿Cuáles han empeorado?

- Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.

### Imports:
A continuación se encuentran todos los imports de las librerias de las que haremos uso en esta práctica.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import operator

## Definición de Funciones

In [2]:
documentos_guardados = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
consultas_a_generar = fetch_20newsgroups(subset='test')

print("Saved documents:", len(documentos_guardados.data))
print("# of consults:", len(consultas_a_generar.data))

Saved documents: 11314
# of consults: 7532


In [3]:
# Crear el diccionario de palabras
with open('Datos/words.txt') as f:
    dictionary = f.read().splitlines()

print("Dictionary Length: ", len(dictionary), " words")

Dictionary Length:  466551  words


In [4]:
def obtenerNombresDeClase(target,consultas_a_generar):
    return (target, consultas_a_generar.target_names[target])

In [5]:
def todosLosTemasTienenNElementos(temas, n):
    for key, value in temas.items():
        if (len(value) != n):
            return False
    return True

In [6]:
def buscarNMensajesPorTema(temas_consultas, mensajes_consultas, n):
    temas = {x: [] for x in consultas_a_generar.target_names}
    i = 0
    while (not todosLosTemasTienenNElementos(temas, n)):
        if (len(temas[temas_consultas[i]]) < n):
            value = (i, mensajes_consultas[i])
            temas[temas_consultas[i]].append(value)
        i += 1
    return temas

In [7]:
def recuperarNResultados(consulta,documentos, n):
    x = cosine_similarity(consulta,documentos)[0]
    y = [index for index in range(len(x))]
    values = list(zip(y,x))
    values.sort(reverse=True, key = operator.itemgetter(1))
    return values[0:n]

In [8]:
def calcularTargets(consulta, documentos, n=3):
    resultado = recuperarNResultados(consulta,documentos, n)
    ids, cosineSimilarity = zip(*resultado)
    targets = []
    for id in ids:
        targets.append(documentos_guardados.target[id])
    
    return targets, cosineSimilarity, ids

In [9]:
def precision(numRelevantDocs, numDocsObtained):
    return numRelevantDocs/numDocsObtained

In [10]:
def getNumRelevantDocs(claseBuscada, clasesObtenidas):
    count = 0
    for clase in clasesObtenidas:
        if (clase == claseBuscada):
            count += 1
    return count

In [11]:
def calcularPrecision(consulta, targetConsulta, documentos, n):
    targets, cosineSimilarity, ids = calcularTargets(consulta, documentos, n)
    ret = precision(getNumRelevantDocs(targetConsulta, targets),n)
    return ret

In [12]:
def generarResultados(nMensajes, topics, documentos, ids, n):
    resultDict = dict()
    for key, values in nMensajes.items():
        resultDict[topics[values[0][0]]] = np.array([])
        for id, value in values:
                resultDict[topics[id]] = np.append(resultDict[topics[id]], calcularPrecision(value, ids[id], documentos, n))
            
    return resultDict

In [13]:
def imprimirResultado(resultados):
    print("Resultados Precisión\n")
    for key, value in resultados.items():
        print("{}: {} -> Mean = {}".format(key,value,value.mean()) )

In [18]:
def ejecutarBuscador(consultas_a_generar, consultas, documentos,n=3):
    id_and_topics = [obtenerNombresDeClase(x,consultas_a_generar) for x in consultas_a_generar.target]
    ids, topics = zip(*id_and_topics)
    nMensajes = buscarNMensajesPorTema(topics, consultas, n)
    resultados = generarResultados(nMensajes, topics, documentos, ids, n)
    imprimirResultado(resultados)

## Apartado A

In [15]:
# Creamos el CountVectorizer

vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english', ngram_range=(1,1), binary=False)
documentos=vectorizer.fit_transform(documentos_guardados.data)

In [16]:
# Tomamos los textos del conjunto de test y los transformamos en una matriz
# de palabras. Al usar "transform" toma como referencia únicamente las palabras
# encontradas en el conjunto de entrenamiento
consultas=vectorizer.transform(consultas_a_generar.data)

In [19]:
ejecutarBuscador(consultas_a_generar,consultas,documentos,3)

Resultados Precisión

alt.atheism: [1. 0. 0.] -> Mean = 0.3333333333333333
comp.graphics: [0.66666667 0.         0.        ] -> Mean = 0.2222222222222222
comp.os.ms-windows.misc: [0.66666667 0.         0.        ] -> Mean = 0.2222222222222222
comp.sys.ibm.pc.hardware: [1.         0.         0.66666667] -> Mean = 0.5555555555555555
comp.sys.mac.hardware: [0. 0. 0.] -> Mean = 0.0
comp.windows.x: [0.         0.33333333 0.66666667] -> Mean = 0.3333333333333333
misc.forsale: [0.66666667 0.66666667 1.        ] -> Mean = 0.7777777777777777
rec.autos: [0. 0. 1.] -> Mean = 0.3333333333333333
rec.motorcycles: [1.         1.         0.66666667] -> Mean = 0.8888888888888888
rec.sport.baseball: [1.         0.66666667 0.66666667] -> Mean = 0.7777777777777777
rec.sport.hockey: [0.33333333 0.66666667 1.        ] -> Mean = 0.6666666666666666
sci.crypt: [1.         0.33333333 0.33333333] -> Mean = 0.5555555555555555
sci.electronics: [0.         0.33333333 1.        ] -> Mean = 0.4444444444444444
sci.med

In [20]:
ejecutarBuscador(consultas_a_generar,consultas,documentos,10)

Resultados Precisión

alt.atheism: [0.7 0.  0.  0.  0.3 0.  0.2 0.2 0.1 0. ] -> Mean = 0.15
comp.graphics: [0.6 0.  0.  0.3 0.6 0.2 0.  0.  0.6 0.4] -> Mean = 0.26999999999999996
comp.os.ms-windows.misc: [0.8 0.1 0.  0.  0.1 1.  0.3 0.3 0.9 0.2] -> Mean = 0.37
comp.sys.ibm.pc.hardware: [1.  0.1 0.4 0.1 0.4 0.2 0.3 0.8 0.3 0.2] -> Mean = 0.38
comp.sys.mac.hardware: [0.1 0.1 0.  0.1 0.4 0.2 0.3 0.1 0.2 0. ] -> Mean = 0.15
comp.windows.x: [0.1 0.4 0.4 0.2 1.  0.5 0.5 0.2 0.1 0.1] -> Mean = 0.35000000000000003
misc.forsale: [0.5 0.6 0.6 0.8 0.9 0.7 0.2 0.5 0.  0.2] -> Mean = 0.5
rec.autos: [0.  0.  0.9 0.3 0.6 0.1 1.  0.9 0.2 0. ] -> Mean = 0.4
rec.motorcycles: [0.9 1.  0.3 1.  1.  0.7 1.  0.  1.  0.7] -> Mean = 0.76
rec.sport.baseball: [1.  0.8 0.5 0.8 0.  0.8 0.6 0.1 0.2 0.6] -> Mean = 0.5399999999999999
rec.sport.hockey: [0.2 0.6 0.9 1.  0.9 0.8 0.8 0.8 0.8 0.9] -> Mean = 0.77
sci.crypt: [0.8 0.2 0.1 0.6 0.3 0.  0.7 0.7 0.1 0.6] -> Mean = 0.41
sci.electronics: [0.1 0.6 0.8 0.3 0.4 0.1 0

**¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?**

`RESPUESTA PENDIENTE`

**Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.**

`RESPUESTA PENDIENTE`

**¿Con qué clases se ha confundido más dicha consulta?**

`RESPUESTA PENDIENTE`

**¿A qué crees que se deben los malos resultados?**

`RESPUESTA PENDIENTE`

## Apartado B

In [21]:
# Creamos el CountVectorizer
tfidfer= TfidfTransformer()

# Calculamos el valor TF-IDF 
documentos_preprocessed=tfidfer.fit_transform(documentos)

In [22]:
# Calculamos el valor TF-IDF 
# Al usar "transform" toma como IDF el del conjunto de entrenamiento 
consultas_preprocessed=tfidfer.transform(consultas)

In [23]:
ejecutarBuscador(consultas_a_generar,consultas_preprocessed,documentos_preprocessed,3)

Resultados Precisión

alt.atheism: [0.66666667 0.         0.        ] -> Mean = 0.2222222222222222
comp.graphics: [0.33333333 0.         0.        ] -> Mean = 0.1111111111111111
comp.os.ms-windows.misc: [0.33333333 0.         0.        ] -> Mean = 0.1111111111111111
comp.sys.ibm.pc.hardware: [1.         0.66666667 0.33333333] -> Mean = 0.6666666666666666
comp.sys.mac.hardware: [0.         0.33333333 0.66666667] -> Mean = 0.3333333333333333
comp.windows.x: [0.         0.66666667 0.33333333] -> Mean = 0.3333333333333333
misc.forsale: [0.33333333 0.66666667 0.33333333] -> Mean = 0.4444444444444444
rec.autos: [0.         0.33333333 1.        ] -> Mean = 0.4444444444444444
rec.motorcycles: [1.         1.         0.33333333] -> Mean = 0.7777777777777778
rec.sport.baseball: [1. 1. 1.] -> Mean = 1.0
rec.sport.hockey: [1. 1. 1.] -> Mean = 1.0
sci.crypt: [1.         0.33333333 0.33333333] -> Mean = 0.5555555555555555
sci.electronics: [0.         0.33333333 1.        ] -> Mean = 0.444444444444444

In [24]:
ejecutarBuscador(consultas_a_generar,consultas_preprocessed,documentos_preprocessed,10)

Resultados Precisión

alt.atheism: [0.7 0.1 0.  0.  0.1 0.  0.4 0.1 0.  1. ] -> Mean = 0.24
comp.graphics: [0.6 0.3 0.  0.4 0.9 0.3 0.  0.  0.2 0.3] -> Mean = 0.3
comp.os.ms-windows.misc: [0.7 0.1 0.  0.  0.2 0.9 0.3 0.3 1.  0.5] -> Mean = 0.4
comp.sys.ibm.pc.hardware: [1.  0.6 0.1 0.5 0.6 0.3 0.3 0.8 0.6 0.2] -> Mean = 0.5
comp.sys.mac.hardware: [0.  0.2 0.4 0.  0.3 0.2 0.4 0.1 0.3 0. ] -> Mean = 0.19
comp.windows.x: [0.1 0.7 0.5 0.  1.  0.2 0.7 0.1 0.1 0.2] -> Mean = 0.36
misc.forsale: [0.5 0.5 0.2 0.8 0.8 0.7 0.1 0.1 0.1 0.2] -> Mean = 0.4
rec.autos: [0.2 0.4 0.9 0.5 0.9 0.  0.9 0.6 0.4 0.5] -> Mean = 0.53
rec.motorcycles: [1.  0.9 0.3 0.9 1.  0.9 1.  0.6 1.  0.6] -> Mean = 0.82
rec.sport.baseball: [1.  0.6 1.  0.9 0.1 0.7 0.8 0.1 1.  1. ] -> Mean = 0.72
rec.sport.hockey: [1.  1.  1.  0.9 1.  0.8 0.8 0.9 1.  1. ] -> Mean = 0.9400000000000001
sci.crypt: [0.9 0.1 0.1 1.  0.3 0.1 1.  1.  0.  1. ] -> Mean = 0.55
sci.electronics: [0.3 0.7 0.8 0.1 0.9 0.1 0.1 0.1 0.6 0. ] -> Mean = 0.37
s

**¿Han cambiado los valores de precisión media para las clases del conjunto de datos?¿Qué clases han mejorado? ¿Cuáles han empeorado?**

`RESPUESTA PENDIENTE`

**Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.**

`RESPUESTA PENDIENTE`